<a href="https://colab.research.google.com/github/abinavrameshs/LLM-Playground/blob/main/Text2SQL_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Why use agents? and not a simple LLM Pipeline?

- LLM pipeline might produce wong SQL and hence wouldnt be able to execute properly.
- Agents can go back and forth until it finds a correct query that executes

In [1]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)



In [13]:
metadata_obj.tables['receipts'].columns.keys()

['receipt_id', 'customer_name', 'price', 'tip']

In [19]:
receipts.c.receipt_id


Column('receipt_id', Integer(), table=<receipts>, primary_key=True, nullable=False)

In [14]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

In [21]:
# Test if values are loaded correctly

from sqlalchemy import select
stmt = select(receipts).where(receipts.c.receipt_id == 1)
print(stmt)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

SELECT receipts.receipt_id, receipts.customer_name, receipts.price, receipts.tip 
FROM receipts 
WHERE receipts.receipt_id = :receipt_id_1
(1, 'Alan Payne', 12.06, 1.2)


In [22]:
## Inspect our table to create description. It will get passed onto System prompt

inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [23]:
columns_info

[('receipt_id', INTEGER()),
 ('customer_name', VARCHAR(length=16)),
 ('price', FLOAT()),
 ('tip', FLOAT())]

In [37]:
from smolagents import tool

def sql_engine(query: str) -> str:
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [43]:
sql_engine.__doc__

In [40]:
sql_engine.__doc__ = \
    f"""
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
    {table_description}

    Args:
        query: The query to perform. This should be correct SQL.
    """

In [41]:
print(sql_engine.__doc__)


    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
    Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    


In [42]:
# Make it as a tool

sql_engine = tool(sql_engine)

In [49]:
from smolagents import CodeAgent, HfApiModel
from google.colab import userdata

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct",token = userdata.get('HF_TOKEN_ALL')),
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1;"                                        
  most_expensive_receipt_name = sql_engine(query=query)                                                            
  print(most_expensive_receipt_name)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 0: Duration 4.65 seconds| Input tokens: 2,128 | Output tokens: 84]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1;"                                        
  most_expensive_receipt_name = sql_engine(query=query)                                                            
  customer_name = most_expensive_receipt_name.strip("()").strip("'")                                               
  final_answer(customer_name)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
('Woodrow Wilson',

[Step 3: Duration 5.77 seconds| Input tokens: 9,777 | Output tokens: 393]

"\n('Woodrow Wilson',"

In [50]:
agent.run("Can you give me the average price and average tips given by all customers? Also tell me who gave the most tip")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the average price and average tips given by all customers? Also tell me who gave the most tip   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Calculate average price and average tip                                                                        
  average_data = sql_engine(query="SELECT AVG(price) as average_price, AVG(tip) as average_tip FROM receipts")     
  print(average_data)                                                                                              
                                                                                                                   
  # Find the customer who gave the most tip                                                                        
  most_tip_customer = sql_engine(query="SELECT customer_name, tip FROM receipts ORDER BY tip DESC LIMIT 1")        
  print(most_tip_customer)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(27.615, 1.9674999999999998)

('Woodrow Wilson', 5.43)

Out: None

[Step 0: Duration 6.94 seconds| Input tokens: 2,135 | Output tokens: 135]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  average_price, average_tip = (27.615, 1.9674999999999998)                                                        
  most_tip_customer, most_tip_amount = ('Woodrow Wilson', 5.43)                                                    
                                                                                                                   
  final_answer(f"The average price is ${average_price:.2f}, the average tip is ${average_tip:.2f}, and the         
  customer who gave the most tip is {most_tip_customer} with a tip of ${most_tip_amount:.2f}.")                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The average price is $27.615, the average tip is $1.9674999999999998, and the customer who gave
the most tip is Woodrow Wilson with a tip of $5.43.

[Step 1: Duration 9.77 seconds| Input tokens: 4,591 | Output tokens: 310]

'The average price is $27.615, the average tip is $1.9674999999999998, and the customer who gave the most tip is Woodrow Wilson with a tip of $5.43.'

In [51]:
agent.run("Who gave he least tip among receipts 2 to 4? What was the price of the product purchased by the customer? ")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Who gave he least tip among receipts 2 to 4? What was the price of the product purchased by the customer?       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT receipt_id, customer_name, price, tip FROM receipts WHERE receipt_id BETWEEN 2 AND 4;"           
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)

Out: None

[Step 0: Duration 4.73 seconds| Input tokens: 2,138 | Output tokens: 92]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer((23.86, 'Alex Mason'))                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: (23.86, 'Alex Mason')

[Step 1: Duration 6.46 seconds| Input tokens: 4,532 | Output tokens: 203]

(23.86, 'Alex Mason')